In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from flask import Flask, render_template, jsonify
import plotly.express as px
import time
import random

# Initialize Flask app
app = Flask(__name__)

# Example threat dataset (threat_id, threat_type, severity, source_ip, timestamp)
# Data will be simulated for demo purposes
def generate_sample_data():
    data = []
    threat_types = ['DDoS', 'Malware', 'Phishing', 'Ransomware', 'Brute Force']
    for i in range(50):
        threat_id = i + 1
        threat_type = random.choice(threat_types)
        severity = random.randint(1, 10)
        source_ip = f'192.168.1.{random.randint(1, 100)}'
        timestamp = time.time() - random.randint(0, 3600)  # random time within the last hour
        data.append({'threat_id': threat_id, 'threat_type': threat_type, 'severity': severity, 'source_ip': source_ip, 'timestamp': timestamp})
    return data

# Preprocess and create features for machine learning
def preprocess_threat_data(data):
    df = pd.DataFrame(data)
    df['time_of_day'] = pd.to_datetime(df['timestamp'], unit='s').dt.hour
    df['threat_type'] = df['threat_type'].astype('category').cat.codes  # Convert threat types to numeric values
    return df

# Train anomaly detection model (Isolation Forest) to detect threats
def train_anomaly_detection_model(df):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[['threat_type', 'severity', 'time_of_day']])
    
    # Using Isolation Forest for anomaly detection
    model = IsolationForest(contamination=0.2)  # Assuming 20% anomalies in data
    model.fit(df_scaled)
    
    # Predict anomalies (1 = normal, -1 = anomaly)
    df['anomaly'] = model.predict(df_scaled)
    return df, model, scaler

# Route to display the threat dashboard
@app.route('/')
def index():
    # Generate simulated threat data
    threat_data = generate_sample_data()
    df = preprocess_threat_data(threat_data)
    df, model, scaler = train_anomaly_detection_model(df)
    
    # Visualize the data using Plotly
    fig = px.scatter(df, x='time_of_day', y='severity', color='anomaly', title='Threat Detection Dashboard',
                     labels={'time_of_day': 'Time of Day', 'severity': 'Threat Severity'},
                     color_discrete_map={1: 'green', -1: 'red'})
    
    # Create a pie chart for threat distribution by type
    pie_fig = px.pie(df, names='threat_type', title='Threat Distribution by Type')
    
    # Convert figures to HTML for embedding in Flask
    scatter_plot_html = fig.to_html(full_html=False)
    pie_chart_html = pie_fig.to_html(full_html=False)
    
    return render_template('dashboard.html', scatter_plot=scatter_plot_html, pie_chart=pie_chart_html)

# Route to get live threat data (AJAX)
@app.route('/live_threat_data')
def live_threat_data():
    threat_data = generate_sample_data()
    df = preprocess_threat_data(threat_data)
    df, model, scaler = train_anomaly_detection_model(df)
    
    live_data = df[['threat_id', 'threat_type', 'severity', 'source_ip', 'anomaly']].to_dict(orient='records')
    return jsonify(live_data)

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)